# Tweet preprocessing code

In [ ]:
#####################################
## Author : Jonathan T.K. Chan
## Created on: 22-04-2020
## Last updated on : 26-04-2020
## Last changed purpose: To preprocess and filter scraped tweets and tweets provided by support team
###################################################

In [1]:
import os
import ast
import pandas as pd
import csv

In [7]:
def tweet_dicts_csv(filename):
    """returns a list of dicts from a csv containing the columns 'tweet_id' and 'tweet_text' for tweets
    (For tweets scraped by project team -  assume RTs and incomplete tweets were removed in initial scraping"""
    max_tweets = 50000
    ind = 0
    tweet_dicts = []
    df = pd.read_csv(filename)

    for index_val, row in df.iterrows(): 
        tweet_dict = {}
        tweet_id = row["id"]
        tweet_text = row["text"]
        tweet_dict["tweet_id"] = tweet_id
        tweet_dict["tweet_text"] = tweet_text
        tweet_dicts.append(tweet_dict)
    
    return tweet_dicts

csv_filename = "racist_processed.csv"
tweet_dicts_csv = tweet_dicts_csv(csv_filename)

In [8]:
def tweet_dicts_txt(filename):
    """ returns a list of dicts from a csv containing the columns 'tweet_id' and 'tweet_text' for tweets
    (for tweets provided by TAs - must remove RTs and incomplete tweets)"""
    max_tweets = 50000
    ind = 0
    tweet_dicts = []
    with open(filename) as f:
        for line in f:
            dict_line = ast.literal_eval(line) #evaluates strings into dicts
            tweet_text = dict_line["tweet_text"]
            if not tweet_text.endswith("...") and not tweet_text.startswith("RT"):
                tweet_dicts.append(dict_line)
                ind += 1
                if ind ==max_tweets:
                    break
    return tweet_dicts

txt_filename = "US_data_about_covid.txt"
tweet_dicts_txt = tweet_dicts_txt(txt_filename)

In [12]:
#filter tweets for tweets that contain any wanted hashtags 
wanted_keywords = set(["chinavirus", "chinesevirus", "chinaliedandpeopledied", 
                       "covidiots", "china","chinese",
                      "chink", "asian", "chinaliedpeopledied", "wuhanvirus", "ccpvirus",
                      "chinesebioterrorism", "boycottchina", "coronavirustruth", "ccp_is_terrorist",
                      "wuflu", "chinesewuhanvirus", 
                      "keepamericagreat", "maga", "makeamericagreatagain", "trump2020", "holdchinaaccountable", "makechinapay", 
                      "racist", "racism",  "ccp",
                      "wholiedpeopledied",  "crime", "propoganda",
                      "communist", "communism"])

def filter_keyword_and_mentions(tweet_dicts, wanted_keywords):
    """returns a list of tweet dicts with the following criteria: 
            -tweet contains mention of word or hashtag related to words in wanted_keywords
            -tweet URL makes up less than 75% of tweet characters"""
    ind = 0
    china_tweets = []
    ht_or_mn_threshold = 0
    for i in range(len(tweet_dicts)):
        tweet = tweet_dicts[i]
        text = tweet["tweet_text"]
        text_tokens = text.split()
        tweet_hashtags = set()
        wanted_tweets = 0
        hashtag_or_mention = 0
        total_tokens = len(text_tokens)
        
        for token in text_tokens:
            if token.startswith("#"):
                hashtag_or_mention += 1
                hashtag_text = token[1:].lower()
                tweet_hashtags.add(hashtag_text)

            if token.startswith("@"):
                hashtag_or_mention += 1

            if token.lower() in wanted_keywords:
                tweet_hashtags.add(token.lower())
                
        wanted_tweets = len(tweet_hashtags & wanted_keywords)
        percent_ht_or_mn = hashtag_or_mention/total_tokens #counts the occurrances of hashtag or mention tokens

        if wanted_tweets > 0:
            if percent_ht_or_mn <=0.75:
                china_tweets.append(tweet)
    
    return china_tweets
    
relevant_tweets = filter_keyword_and_mentions(tweet_dicts_txt, wanted_keywords)

In [13]:


def filter_url(relevant_tweets):
    """returns a list of tweet dicts where tweet URL makes up less than 50% of tweet characters"""
    ind = 0
    link_threshold = 0.5

    final_tweets = []

    for i in range(len(relevant_tweets)):
        tweet = relevant_tweets[i]
        text = tweet["tweet_text"]
        text_tokens = text.split()

        text_tokens = [x.lower() for x in text_tokens]      
        link_token = ""

        for token in text_tokens:
            if token.startswith("https://"):
                link_token += token
                link_percentage = len(link_token)/len(text)
                if link_percentage <= 0.5:
                    final_tweets.append(tweet)


    return final_tweets

final_tweets = filter_url(china_tweets)

In [14]:
def remove_duplicates(final_tweets):
    """returns a list of tweet dicts where tweet content (characters before the tweet URL) have not repeated"""
    ind = 0
    added_text = set()
    csv_tweets = []
    for i in range(len(final_tweets)):
        tweet = final_tweets[i]
        tweet_text = tweet["tweet_text"]
        text_content = tweet_text.split("https://")[0]
        if text_content not in added_text:
            added_text.add(text_content)
            csv_tweets.append(tweet)
    
    return csv_tweets

csv_tweets = remove_duplicates(final_tweets)  

In [17]:
def tweets_to_csv(csv_tweets):
    """"""
    ind = 0
    #output_filename = "covid_tweets.csv"
    max_count = 500
    output_filename = "racist_preprocessed_filtered_" + str(max_count) + ".csv"
    
    header = 'tweet_id, tweet_text'

    with open(output_filename, "w") as file:
        writer = csv.writer(file)

        writer.writerow(["tweet_id","tweet_text"])
        for i in range(len(csv_tweets)):
            tweet = csv_tweets[i]
            tweet_id = tweet["tweet_id"]
            tweet_text = tweet["tweet_text"]
            try:
                writer.writerow([tweet_id, tweet_text])
                ind += 1
            except:
                pass
            if ind == max_count:
                break

tweets_to_csv(csv_tweets)